
# STRUCTURE

In [3]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [4]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [5]:
# the path to your HDF5 formatted snps file
data = "./G_cy_5rm_v9.snps.hdf5"

In [6]:
imap = {
#"ref": ["reference"],
#"Inam": ["G_cy_T12385_In","G_cy_T310_In","G_cy_T23196_In","G_cy_T7636_In"],
#"Puru": ["G_cy_82508_pu","G_cy_T12275_pu","G_cy_T12279_pu","G_cy_T12392_pu","G_cy_T13184_pu","G_cy_T26228_pu","G_cy_T26229_pu","G_cy_T26252_pu"],
#"JiGu": ["G_cy_T3343_jigu","G_cy_T3384_jigu","G_cy_T3385_jigu"],
"Mach": ["G_cy_J296_ma","G_cy_J477_ma","G_cy_J773_ma","G_cy_T13251_ma","G_cy_T363_ma","G_cy_T364_ma"],
"Roar": ["G_cy_J691_roar","G_cy_J694_roar"],
"ArSu": ["G_cy_80582_arsu","G_cy_85678_arsu","G_cy_80701_arsu","G_cy_80801_arsu","G_cy_80826_arsu","G_cy_81108_arsu","G_cy_81118_arsu","G_cy_85499_arsu"],
"SuTa": ["G_cy_85356_suta","G_cy_86297_suta","G_cy_86321_suta","G_cy_86458_suta","G_cy_86478_suta","G_cy_T14558_suta","G_cy_T16693_suta","G_cy_T18563_suta","G_cy_T18620_suta"],
"Para": ["G_cy_T10897_pa","G_cy_T11062_pa","G_cy_T16771_pa","G_cy_T1705_pa","G_cy_T18744_pa","G_cy_T19429_pa","G_cy_T19520_pa","G_cy_T19765_pa","G_cy_T2497_pa","G_cy_T6579_pa","G_cy_T9133_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="G_cy_str_east",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 36
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 15257
Filtered (mincov): 1054810
Filtered (minmap): 1100681
Filtered (combined): 1101162
Sites after filtering: 29375
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000
#struct.write_structure_files(abs)

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 3:44:04 | running 70 structure jobs 


In [8]:
struct = ipa.structure(
    data=data, 
    name="G_cy_str_east", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [G_cy_str_east]


In [10]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-22199.22,328.164,0.00,0.00
3,10,1.272,-20854.40,1466.701,1344.82,1865.14
4,10,1.566,-21374.72,454.242,-520.32,711.49
5,10,78.932,-21183.55,370.581,191.17,29250.80
6,10,0.623,-50243.18,91503.878,-29059.63,56975.39
7,10,27.050,-22327.42,1073.957,27915.76,29050.78
8,10,0.000,-23462.44,2710.024,-1135.02,0.00


In [11]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="ta09baddff882436db00645e07bb48533" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 20000 30000 40000 50000 estLnProbMean 0 50 100 deltaK

In [12]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [13]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [14]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tb7f2115b18964e409a82379dd7950826" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

In [15]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [16]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [17]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1fb6d7e76fd34f4d8966e7985a942ad4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

In [18]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [19]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [20]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9f5471c7343b4d378290da3c6af7e1f1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

In [34]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [35]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [36]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t594c405298324072b4795be3887e6c0b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

In [30]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [31]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [32]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t7e9fb3e560e14404aab2c64361d204d2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

In [33]:
k = 7
table = struct.get_clumpp_table(k)
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K7] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t8e6d9a4e118c43cea87a7d25b003b5e6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma 0.0 0.5 1.0

# PCA

In [66]:
#!conda install scikit-learn -c conda-forge -y

In [7]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [8]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 36
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 15257
Filtered (mincov): 588913
Filtered (minmap): 1100681
Filtered (combined): 1101162
Sites after filtering: 29375
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [9]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 4084/29375


In [10]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b728ebdae90>,
 <toyplot.mark.Point at 0x2b728e044f10>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t89e76c5fa0f54fbea845f5418b524b3d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T363_ma G_cy_T364_ma G_cy_T6579_pa G_cy_T9133_pa -10 -5 0 5 10 PC0 (27.8%) explained -5 0 5 10 PC1 (14.3%) explained Roar ArSu SuTa Para Mach

In [11]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("G_cy_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
G_cy_80582_arsu,-8.37,-1.03,-1.88,7.74,5.23,-1.16,4.13,-0.77,2.05,-5.24
G_cy_80701_arsu,-8.05,-1.56,1.18,1.19,0.57,0.97,-2.27,3.91,-2.12,-1.58
G_cy_80801_arsu,-8.32,-1.46,5.09,1.08,-2.60,1.62,-0.33,1.09,0.54,0.17
G_cy_80826_arsu,-8.57,-1.96,0.12,-1.20,1.87,1.55,-1.12,0.23,-1.00,1.27
G_cy_81108_arsu,-8.20,-1.32,4.44,0.94,-0.69,2.59,5.25,-3.41,-1.09,-2.48
G_cy_81118_arsu,-8.05,-1.12,-2.46,-0.98,2.69,-4.94,-3.74,-4.66,-1.64,0.86
G_cy_85356_suta,-8.27,-0.75,0.27,-2.58,-4.82,-0.46,-2.47,-2.81,-4.00,-3.24
G_cy_85499_arsu,-7.70,-1.74,-0.60,0.92,1.93,-0.82,-2.36,0.28,-1.75,-1.98
G_cy_85678_arsu,-8.24,-0.97,-1.17,-2.19,-4.09,-4.41,2.48,-4.03,6.23,-0.85
G_cy_86297_suta,-8.09,-1.78,-0.18,-0.31,0.49,1.44,-0.82,2.03,-3.05,0.23


# Subsampling with replication

In [12]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 2);

Subsampling SNPs: 4084/29375


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3cb618a32dc14518ad8acd7f1ace1de5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T363_ma <g class="toyplot-Datum" style="fill:rgb(62%,0.4%,25.9%);fill-opacity:0.75;opacity:1.0;stroke:rgb(14.9%,14.9%,14

# Kmeans imputation (integer)

In [13]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 2);

Subsampling SNPs: 4084/29375


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t28e07f94024245c5bca5c76fb7433dfd" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T363_ma G_cy_T364_ma <g class="toyplot-Datum" style=

In [14]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [15]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [16]:
pca.run_tsne(subsample=True, perplexity=8.0, n_iter=100000, seed=123)

Subsampling SNPs: 4084/29375


In [17]:
pca.draw(size=10);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tda2573ac3be24970be86497979f90361" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T363_ma G_cy_T364_ma G_cy_T6579_pa G_cy_T9133_pa -250 0 250 TNSE component 1 -300 -200 -100 0 100 200 TNSE component 2 Roar ArSu SuTa Para Mach

In [18]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_east_TSNE_perp8.pdf")

# GENETIC DISTNANCES

In [19]:
import ipyrad.analysis as ipa
import toyplot

In [20]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 36
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 15257
Filtered (mincov): 588913
Filtered (minmap): 1100681
Filtered (combined): 1101162
Sites after filtering: 29375
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [21]:
# save to a CSV file
dist.dists.to_csv("G_cy_distances.csv")

In [22]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "G_cy_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [23]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [24]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t4e1eeeb673c9467bac3b78d18b8c3ce5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.084528 0.141447 0.145123 0.145157 0.151217 0.150264 0.144511 0.147711 0.143387 0.146111 0.142638 0.145736 0.146213 0.149243 0.147234 0.151694 0.146962 0.146860 0.133617 0.135489 0.135557 0.139949 0.133311 0.133277 0.134468 0.137260 0.136477 0.135115 0.136374 0.081430 0.081123 0.084426 0.077072 0.080647 0.075983 G_cy_J691_roar 1 0.084528 0.000000 0.147064 0.145566 0.148664 0.153157 0.151183 0.146111 0.151149 0.145668 0.150774 0.146826 0.148766 0.147132 0.149209 0.151081 0.154587 0.153328 0.146145 0.138281 0.138996 0.141038 0.146723 0.140766 0.138757 0.139609 0.141379 0.140596 0.140596 0.140153 0.087115 0.085923 0.089498 0.083983 0.083404 0.088340 G_cy_J694_roar 2 0.141447 0.147064 0.000000 0.082247 0.083166 0.084936 0.086366 0.082315 0.084017 0.079489 0.082553 0.084119 0.085991 0.084766 0.086230 0.084630 0.087591 0.083200 0.086162 0.147336 0.149889 0.152681 0.155574 0.148528 0.149447 0.150502 0.150570 0.150877 0.151149 0.151660 0.142400 0.144000 0.147098 0.140153 0.147064 0.144034 G_cy_80582_arsu 3 0.145123 0.145566 0.082247 0.000000 0.081464 0.084868 0.084187 0.080681 0.082655 0.076426 0.079898 0.082009 0.081294 0.081566 0.083847 0.082996 0.082553 0.081770 0.083983 0.152443 0.153974 0.156221 0.157345 0.152885 0.152511 0.152545 0.152885 0.152034 0.152511 0.153974 0.146281 0.146928 0.150230 0.145464 0.146587 0.148460 G_cy_85678_arsu 4 0.145157 0.148664 0.083166 0.081464 0.000000 0.080409 0.083132 0.081191 0.082894 0.080136 0.081294 0.083064 0.080783 0.081736 0.085719 0.083711 0.084357 0.082145 0.082043 0.150774 0.155370 0.157004 0.158604 0.153123 0.151932 0.154417 0.151830 0.154519 0.156834 0.155779 0.147200 0.147574 0.150332 0.144409 0.148868 0.148834 G_cy_80701_arsu 5 0.151217 0.153157 0.084936 0.084868 0.080409 0.000000 0.085447 0.081736 0.087864 0.083472 0.085515 0.087081 0.086230 0.087251 0.088170 0.086094 0.087081 0.084187 0.085515 0.155200 0.157821 0.160000 0.163234 0.155574 0.154655 0.157140 0.154826 0.155404 0.158264 0.157889 0.152374 0.152613 0.155711 0.149787 0.154519 0.154145 G_cy_80801_arsu 6 0.150264 0.151183 0.086366 0.084187 0.083132 0.085447 0.000000 0.085345 0.082485 0.076732 0.081157 0.084221 0.085140 0.085004 0.084902 0.084187 0.084426 0.082826 0.082247 0.154043 0.154553 0.160409 0.162621 0.153260 0.152613 0.153872 0.154553 0.155540 0.155064 0.154281 0.152374 0.150570 0.153872 0.147949 0.152068 0.152987 G_cy_80826_arsu 7 0.144511 0.146111 0.082315 0.080681 0.081191 0.081736 0.085345 0.000000 0.082451 0.080919 0.083711 0.082826 0.082111 0.083745 0.085753 0.085447 0.083915 0.080272 0.085957 0.150877 0.153362 0.156289 0.158162 0.151796 0.151149 0.154655 0.151251 0.152715 0.153940 0.154247 0.146689 0.145294 0.149140 0.142740 0.148426 0.147915 G_cy_81108_arsu 8 0.147711 0.151149 0.084017 0.082655 0.082894 0.087864 0.082485 0.082451 0.000000 0.079694 0.082757 0.082894 0.085923 0.081634 0.085413 0.082655 0.087047 0.080340 0.082009 0.151217 0.154179 0.157174 0.158843 0.152885 0.151694 0.154587 0.154179 0.154553 0.154689 0.152817 0.150502 0.146928 0.152545 0.145804 0.150264 0.151796 G_cy_81118_arsu 9 0.143387 0.145668 0.079489 0.076426 0.080136 0.083472 0.076732 0.080919 0.079694 0.000000 0.079319 0.079728 0.082553 0.079557 0.082247 0.081872 0.083677 0.080851 0.078843 0.146553 0.147745 0.152579 0.153634 0.147200 0.146826 0.146860 0.148153 0.147711 0.150230 0.148357 0.146179 0.143353 0.148698 0.143387 0.144783 0.146315 G_cy_85499_arsu 10 0.146111 0.150774 0.082553 0.079898 0.08